In [1]:
%load_ext autoreload
%autoreload 2

# Step loading data

In [1]:
import pandas as pd
import numpy as np

In [3]:
from openproblems.tasks.regulatory_effect_prediction import datasets
adata = datasets.share_mouse_skin(test=False)
adata.shape

(34774, 23076)

In [5]:
adata.uns['mode2_var_chr'].shape, adata.uns['mode2_var'].shape

((344592,), (344592, 3))

In [7]:
adata.shape, adata.obsm['mode2'].shape

((34774, 23076), (34774, 344592))

## Copy of utility functions

In [9]:
from openproblems.patch import patch_datacache
import numpy as np
import pandas as pd
import scanpy as sc
import warnings

def _chrom_limit(x, tss_size=2e5):
    """Extend TSS to upstream and downstream intervals.

    Parameters
    ----------
    x : pd.Series
        a pd.Series containing [start, end, direction]
        where start and end are ints and direction is {'+', '-'}.
    tss_size: int
        a int that defines the upstream and downstream regions around TSS
    """
    y = x.values
    gene_direction = y[-1]
    gene_start = y[-3]
    gene_end = y[-2]
    if gene_direction == "+":
        return [gene_start - tss_size // 2, gene_start + tss_size // 2]
    else:
        return [gene_end - tss_size // 2, gene_end + tss_size // 2]


def _get_annotation(adata, retries=3):
    """Insert meta data into adata.obs."""
    from pyensembl import EnsemblRelease

    data = EnsemblRelease(
        adata.uns["release"],
        adata.uns["species"],
    )
    for _ in range(retries):
        try:
            with patch_datacache():
                data.download(overwrite=False)
                data.index(overwrite=False)
            break
        except TimeoutError:
            pass

    # get ensemble gene coordinate
    genes = []
    for i in adata.var.index.map(lambda x: x.split(".")[0]):
        try:
            with warnings.catch_warnings():
                warnings.filterwarnings(
                    action="ignore", message="No results found for query"
                )
                gene = data.gene_by_id(i)
            genes.append(
                [
                    "chr%s" % gene.contig,
                    gene.start,
                    gene.end,
                    gene.strand,
                ]
            )
        except ValueError:
            try:
                with warnings.catch_warnings():
                    warnings.filterwarnings(
                        action="ignore", message="No results found for query"
                    )
                    i = data.gene_ids_of_gene_name(i)[0]
                gene = data.gene_by_id(i)
                genes.append(
                    [
                        "chr%s" % gene.contig,
                        gene.start,
                        gene.end,
                        gene.strand,
                    ]
                )
            except (IndexError, ValueError) as e:
                print(e)
                genes.append([np.nan, np.nan, np.nan, np.nan])
    old_col = adata.var.columns.values
    adata.var = pd.concat(
        [adata.var, pd.DataFrame(genes, index=adata.var_names)], axis=1
    )
    adata.var.columns = np.hstack(
        [old_col, np.array(["chr", "start", "end", "strand"])]
    )


def _filter_mitochondrial(adata):
    if adata.uns["species"] in ["mus_musculus", "homo_sapiens"]:
        adata.var["mt"] = adata.var.gene_short_name.str.lower().str.startswith(
            "mt-"
        )  # annotate the group of mitochondrial genes as 'mt'
        sc.pp.calculate_qc_metrics(
            adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
        )

        adata_filter = adata[adata.obs.pct_counts_mt <= 10]
        if adata_filter.shape[0] > 100:
            adata = adata_filter.copy()
    return adata


def _filter_n_genes_max(adata):
    adata_filter = adata[adata.obs.n_genes_by_counts <= 2000]
    if adata_filter.shape[0] > 100:
        adata = adata_filter.copy()
    return adata


def _filter_n_genes_min(adata):
    adata_filter = adata.copy()
    sc.pp.filter_cells(adata_filter, min_genes=200)
    if adata_filter.shape[0] > 100:
        adata = adata_filter
    return adata


def _filter_n_cells(adata):
    adata_filter = adata.copy()
    sc.pp.filter_genes(adata_filter, min_cells=5)
    if adata_filter.shape[1] > 100:
        adata = adata_filter
    return adata


def _filter_has_chr(adata):
    adata_filter = adata[:, ~pd.isnull(adata.var.loc[:, "chr"])].copy()
    if adata_filter.shape[1] > 100:
        adata = adata_filter
    return adata

## Get gene coordinates

In [10]:
_get_annotation(adata)

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['0610007P14Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['0610009O20Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['0610011F06Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['0610037L13Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['1010001N08Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['1110001J03Rik']
No results found for q

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['A130057D12Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['A230046K03Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['A230050P20Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['A230065H16Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['A330017A19Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['A330050F15Rik']
No results found for q

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['B130006D01Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['B230344G16Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['B630005N14Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['B930041F14Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['B930082K07Rik']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['BC003331']
No results found for query:

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Cml1']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Cml3']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Cml5']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Cpsf3l']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Csrp2bp']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Ctage5']
No results found for query:

            SELECT distinct gene_id
    

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Gm10020']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Gm10031']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Gm10083']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Gm10093']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Gm10094']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Gm10104']
No results found for query:

            SELECT distinct g

...

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Hist1h1a']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Hist1h1b']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Hist1h1c']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Hist1h1d']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Hist1h1e']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Hist1h1t']
No results found for query:

            SELECT dist

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Lnp']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Lppos']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Lrrc16a']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Lrrc16b']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Lrrc48']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Lyrm5']
No results found for query:

            SELECT distinct gene_id
  

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Pla2g16']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Platr17']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Platr2']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Ppp1r3fos']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Ppp2r4']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Pqlc1']
No results found for query:

            SELECT distinct gen

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Rfwd2']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Rgag1']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Rgag4']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Ric8']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Rltpr']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Rnmtl1']
No results found for query:

            SELECT distinct gene_id
     

No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Tusc5']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Vimp']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Vprbp']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['Vwa9']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['WDR97']
No results found for query:

            SELECT distinct gene_id
            FROM gene
            WHERE gene_name = ?
        
with parameters: ['WI1-2587D24']
No results found for query:

            SELECT distinct gene_id
 

In [11]:
adata.var.head()

,gene_short_name,n_counts,chr,start,end,strand
0610007P14Rik,0610007P14Rik,485.0,NaN,NaN,NaN,NaN
0610009B22Rik,0610009B22Rik,133.0,chr11,51685386.0,51688874.0,-
0610009L18Rik,0610009L18Rik,53.0,chr11,120348678.0,120351190.0,+
0610009O20Rik,0610009O20Rik,750.0,NaN,NaN,NaN,NaN
0610010F05Rik,0610010F05Rik,1570.0,chr11,23564961.0,23633639.0,-


In [12]:
adata.shape, adata.obsm['mode2'].shape

((34774, 23076), (34774, 344592))

In [13]:
# basic quality control
adata = _filter_has_chr(adata)
adata = _filter_mitochondrial(adata)
adata = _filter_n_genes_max(adata)
adata = _filter_n_genes_min(adata)
adata = _filter_n_cells(adata)

In [14]:
adata.shape, adata.obsm['mode2'].shape

((33827, 18671), (33827, 344592))

## Normalize 

In [15]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# (Joint) Feature selection 

* currently selected by top 2000 variable genes

In [16]:
if 2000 <= adata.shape[1]:
    sc.pp.highly_variable_genes(adata, n_top_genes=2000)
    adata = adata[:, adata.var.highly_variable].copy()

In [17]:
# generate peak to gene weight matrix
# remove genes without annotation
adata = adata[
    :, adata.var.loc[:, "chr"].isin(np.unique(adata.uns["mode2_var_chr"]))
].copy()

In [18]:
adata.shape, adata.obsm['mode2'].shape

((33827, 1994), (33827, 344592))

# Regulatory effect score

In [19]:
# filter atac-seq matrix
sel = np.isin(adata.uns["mode2_var_chr"], adata.var.loc[:, "chr"].unique())
adata.uns["mode2_var"] = adata.uns["mode2_var"][sel]
adata.uns["mode2_var_chr"] = adata.uns["mode2_var_chr"][sel]
adata.uns["mode2_var_start"] = adata.uns["mode2_var_start"][sel]
adata.uns["mode2_var_end"] = adata.uns["mode2_var_end"][sel]
adata.obsm["mode2"] = adata.obsm["mode2"][:, sel]

In [21]:
adata.shape, adata.obsm['mode2'].shape

((33827, 1994), (33827, 344592))

In [22]:
# extend tss upstream and downstream
extend_tss = adata.var.loc[:, ["start", "end", "strand"]].apply(
    _chrom_limit, axis=1
)
extend_tss.shape

(1994,)

In [23]:
if isinstance(extend_tss, pd.DataFrame):
    # should be a series
    extend_tss = extend_tss.iloc[:, 0]

extend_tss = pd.concat(
    [
        adata.var.loc[:, "chr"],
        extend_tss.map(lambda x: x[0]).astype("int32"),
        extend_tss.map(lambda x: x[1]).astype("int32"),
        pd.Series(np.arange(adata.shape[1]), index=adata.var_names),
    ],
    axis=1,
)

In [24]:
(extend_tss.iloc[:, 1]>0).sum(), (extend_tss.iloc[:, 2]>0).sum()

(1994, 1994)

In [25]:
# peak summits
peaks = pd.DataFrame(
    {
        "chr": adata.uns["mode2_var_chr"],
        "start": adata.uns["mode2_var_start"].astype("int32"),
        "end": adata.uns["mode2_var_end"].astype("int32"),
    }
)
summits = pd.concat(
    [
        peaks.iloc[:, 0],
        peaks.iloc[:, [1, 2]].mean(axis=1).astype("int32"),
        (peaks.iloc[:, [1, 2]].mean(axis=1) + 1).astype("int32"),
        pd.Series(np.arange(peaks.shape[0])),
    ],
    axis=1,
)

In [26]:
peaks.shape, summits.shape

((344592, 3), (344592, 4))

## Intersection between peak and TSS(+/- 100kb)

In [27]:
import pybedtools

# overlap TSS bins with peaks
x = pybedtools.BedTool.from_dataframe(summits)
y = pybedtools.BedTool.from_dataframe(extend_tss)

In [28]:
#tss_to_peaks = y.intersect(x, wb=True, wa=True, loj=False).to_dataframe()
tss_to_peaks = x.intersect(y, wb=True, wa=True, loj=True)

/Users/qq06/anaconda3/envs/openproblem/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [29]:
tss_to_peaks = tss_to_peaks.to_dataframe(disable_auto_names=True, header=0,
                                         names=["peak_chr", 
                                                 "peak_summit_left",
                                                 "peak_summit_right", 
                                                 "peak_index", 
                                                 "gene_chr", 
                                                 "gene_bin_start", 
                                                 "gene_bin_end", 
                                                 "gene_index"])
print(tss_to_peaks.shape)

(362657, 8)


In [30]:
tss_to_peaks.head()

,peak_chr,peak_summit_left,peak_summit_right,peak_index,gene_chr,gene_bin_start,gene_bin_end,gene_index
0,chr6,3201126,3201127,1,.,-1,-1,.
1,chr9,123462000,123462001,2,chr9,123266927,123466927,1203
2,chr1,56782245,56782246,3,.,-1,-1,.
3,chr9,56223818,56223819,4,.,-1,-1,.
4,chr1,88277609,88277610,5,.,-1,-1,.


In [31]:
# remove non-overlapped TSS and peaks
tss_to_peaks = tss_to_peaks.loc[
    (tss_to_peaks.gene_chr != ".") | (tss_to_peaks.gene_index != "."), :
]

In [34]:
tss_to_peaks.shape, (tss_to_peaks.iloc[:, 1] >0).sum(), (tss_to_peaks.iloc[:, 2] >0).sum()

((97429, 8), 97429, 97429)

In [33]:
tss_to_peaks.shape, (tss_to_peaks.iloc[:, 5] >0).sum(), (tss_to_peaks.iloc[:, 6] >0).sum()

((97429, 8), 97429, 97429)

In [36]:
tss_to_peaks.apply(
    lambda x: abs((int(x[5]) + int(x[6])) / 2 - int(x[1])) * 1.0 / 1e5, axis=1
).values

array([0.95073, 0.87698, 0.29745, ..., 0.0534 , 0.90248, 0.763  ])

In [37]:
tss_to_peaks.shape

(97429, 8)

## Peak to gene distance weighted by MARGE

In [38]:
tss_to_peaks.loc[:, "distance"] = tss_to_peaks.apply(
    lambda x: abs((int(x[5]) + int(x[6])) / 2 - int(x[1])) * 1.0 / 1e5, axis=1
)

In [39]:
alpha = -np.log(1.0 / 3.0) * 1e5 / 1e4
alpha    

10.986122886681098

In [40]:
decay_score = np.exp(-alpha * tss_to_peaks["distance"].values)
tss_to_peaks["weight"] = 2.0 * decay_score / (1.0 + decay_score)

In [41]:
tss_to_peaks["weight"].shape

(97429,)

In [42]:
(tss_to_peaks["weight"]>0.9).sum()

5141

In [43]:
import scipy
gene_peak_weight = scipy.sparse.csr_matrix(
    (
        tss_to_peaks.weight.values,
        (tss_to_peaks.gene_index.astype("int32").values, tss_to_peaks.peak_index),
    ),
    shape=(adata.shape[1], adata.uns["mode2_var"].shape[0]),
)

In [44]:
adata.obsm["gene_score"] = adata.obsm["mode2"] @ gene_peak_weight.T

# Imputation of single cell RNA-seq

In [45]:
import scanpy as sc

In [46]:
sc.pp.pca(adata)

In [47]:
sc.pp.neighbors(adata, n_neighbors=300)

In [48]:
adata.layers['X_knn'] = adata.obsp['connectivities'].dot(adata.X)

# Imputatation of single cell ATAC-seq

In [50]:
adata.layers['gene_score_knn'] = adata.obsp['connectivities'].dot(adata.obsm["gene_score"])

# Metrics

## Across genes

In [57]:
def compute_similarity2(O, P):
    "Compute pearson correlation between two matrices O and P using einstein summation"
    (n, t) = O.shape      # n traces of t samples
    (n_bis, m) = P.shape  # n predictions for each of m candidates

    DO = O - (np.einsum("nt->t", O, optimize='optimal') / np.double(n)) # compute O - mean(O)
    DP = P - (np.einsum("nm->m", P, optimize='optimal') / np.double(n)) # compute P - mean(P)

    cov = np.einsum("nm,nt->mt", DP, DO, optimize='optimal')

    varP = np.einsum("nm,nm->m", DP, DP, optimize='optimal')
    varO = np.einsum("nt,nt->t", DO, DO, optimize='optimal')
    tmp = np.einsum("m,t->mt", varP, varO, optimize='optimal')
    return cov / np.sqrt(tmp)

In [59]:
cors = compute_similarity2(adata.layers['X_knn'].toarray(), adata.layers['gene_score_knn'].toarray())

<ipython-input-57-3519e1864505>:14: RuntimeWarning: invalid value encountered in true_divide
  return cov / np.sqrt(tmp)


In [63]:
np.median(cors[~np.isnan(cors)])

0.07538634116228767

In [66]:
cors_diag = np.diag(cors)
np.median(cors_diag[~np.isnan(cors_diag)])

0.14733736751645934

## Across pseudo-bulk cell aggregates by KNN

In [72]:
cors = compute_similarity2(adata.layers['X_knn'].toarray().T[:, :10000], 
                           adata.layers['gene_score_knn'].toarray().T[:, :10000])

In [73]:
np.median(cors[~np.isnan(cors)])

0.13612813178507155

In [74]:
cors_diag = np.diag(cors)
np.median(cors_diag[~np.isnan(cors_diag)])

0.18688686100935356

## Global correlation

In [53]:
def _global_pearson(x, y):
    numerator = np.mean((x - x.mean()) * (y - y.mean()))
    denominator = x.std() * y.std()
    if denominator == 0:
        return 0
    else:
        result = numerator / denominator
        return result

In [54]:
_global_pearson(adata.layers['X_knn'].toarray(),
                adata.layers['gene_score_knn'].toarray())

0.2139866102030402